<a href="https://colab.research.google.com/github/Anjan-AI/TSAI-EVA5/blob/master/Session%204%20-%20Architectural%20Basics/EVA5_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing all the Python Packages & torch Library.
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            
            nn.Conv2d(1, 8, 3), # output 26X26X8 : RF- 3X3
            nn.ReLU(),
            nn.BatchNorm2d(8), # Batch Normalization after each convolution.
            nn.Dropout2d(0.1), # dropout of 10% at each layer
            
            nn.Conv2d(8, 16, 3), # output 24X24X16 : : RF- 5X5
            nn.ReLU(),
            nn.BatchNorm2d(16), # Batch Normalization after each convolution.
            nn.Dropout2d(0.1),  # dropout of 10% at each layer

            nn.Conv2d(16, 32, 3), # output 22X22X32 : RF- 7X7
            nn.ReLU(),
            nn.BatchNorm2d(32),  # Batch Normalization after each convolution.
            nn.Dropout2d(0.1),  # dropout of 10% at each layer

           # using 1X1 filter to reduce the no of channel.
            nn.Conv2d(32, 8, 1), # output 22X22X8 : RF- 7X7
            nn.ReLU(),
            nn.BatchNorm2d(8), # Batch Normalization after each convolution.
            nn.Dropout2d(0.1), # dropout of 10% at each layer
            # hitting Max pooling when RF is 7X7
            nn.MaxPool2d(2, 2)       # output 11X11X8 : RF- 14X14
            )
        self.conv2 = nn.Sequential(
            
            nn.Conv2d(8, 16, 3), # output 09X09X16 : RF- 16 X16
            nn.ReLU(),
            nn.BatchNorm2d(16), # Batch Normalization after each convolution.
            nn.Dropout2d(0.1),  # dropout of 10% at each layer
            
            nn.Conv2d(16, 32, 3), # output 07X07X32 : RF- 18 X18
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
           
           # using 1X1 filter to reduce the no of channel.
            nn.Conv2d(32, 8, 1), # output 07X07X8 : RF- 18 X18
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.1),
           # nn.MaxPool2d(2, 2)    
            )
        self.conv3 = nn.Sequential(
           
            nn.Conv2d(8, 16, 3), # output 05X05X16 : RF- 20 X20
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            # used padding so that the image size reamin to be 5 X5 
            nn.Conv2d(16, 32, 3,padding=1), # output 05X05X32 : RF- 22 X 22
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),

            nn.Conv2d(32, 10, 1), # output 05X05X10 : RF- 22 X 22
           # nn.ReLU(),           # no Relu or BN near to putput layer
           # nn.BatchNorm2d(10),

            #nn.Conv2d(16, 10, 1), #22
           # nn.BatchNorm2d(10),
            nn.AvgPool2d(5)

            )
        
    def forward(self, x):
        
        x = self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
         Dropout2d-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,168
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,640
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
           Conv2d-13            [-1, 8, 22, 22]             264
             ReLU-14            [-1, 8,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
# seed the model to obtain consistent results
torch.manual_seed(1)
# this is the batch size , in 1 pas no of images passed together.
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# load the training data and perform standard normalization 
# parameter for normalization is mean and std dev.
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
#load the test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
# Function to train 
'''
Args: 
Model : created model to be used for training
device : GPU or cpu
train_laoded: data on which the training has to be done
Optimizer : the optimization algorithm to be used
epoch : no fo epoch 

'''
def train(model, device, train_loader, optimizer, epoch):
    model.train() # Set the model on training mode
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # moving the data to device
        optimizer.zero_grad() # zero the graidents 
        output = model(data) # getting the model output
        loss = F.nll_loss(output, target) # calculating the The negative log likelihood loss
        loss.backward() # flowing the gradients backward.
        optimizer.step() # paameter updated basd on the current gradient.
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

# Function to test 
'''
Args: 
Model : created model to be used for training
device : GPU or cpu
test_laoded: data on which the testing has to be done
 

'''
def test(model, device, test_loader):
    model.eval() # seting up the model for evalaution.
    test_loss = 0 # setting the test loss to 0
    correct = 0 # countign the no of coorect classfication.
    with torch.no_grad(): # turn off gradients, since we are in test mode
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)   # copy the data to device.
            output = model(data) # predict the output
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset) # calculating hte test loss.

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
 
model = Net().to(device) # move the model to device.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # intiating the SGD optimizer

for epoch in range(1, 21):
    print("epoch =", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    

  0%|          | 0/469 [00:00<?, ?it/s]

epoch = 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0754, Accuracy: 9780/10000 (97.80%)

epoch = 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0211, Accuracy: 9935/10000 (99.35%)

epoch = 19


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0215, Accuracy: 9936/10000 (99.36%)

epoch = 20


loss=0.016440650448203087 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.41it/s]



Test set: Average loss: 0.0212, Accuracy: 9935/10000 (99.35%)

